In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata
import sys

sys.path.append('../')
import preprocessing_tools as pt

In [2]:
def label_cd4_cd8_from_markers(
    adata,
    t_mask=None,                  # boolean mask for T cells; if None, uses all cells
    layer="counts",               # use raw counts if present
    out_col="cd4_cd8",
    delta=0.25,                   # required separation between CD4 and CD8 scores
    min_score=0.10,               # require at least some signal
):
    # work on a copy to avoid touching your main object (no saving)
    ad = adata.copy()

    # choose expression matrix
    if layer is not None and layer in ad.layers:
        ad.X = ad.layers[layer].copy()

    # log-normalize for scoring
    sc.pp.normalize_total(ad, target_sum=1e4)
    sc.pp.log1p(ad)

    # common markers (gene SYMBOLS)
    t_markers   = ["TRAC", "CD3D", "CD3E"]  # optional sanity check
    cd4_markers = ["CD4", "IL7R", "CCR7", "LTB", "TCF7", "LEF1"]  # MALAT1 is ubiquitous; remove if you dislike
    cd8_markers = ["CD8A", "CD8B"]

    # keep only genes present
    def present(gs): return [g for g in gs if g in ad.var_names]
    cd4g, cd8g = present(cd4_markers), present(cd8_markers)
    if len(cd4g) == 0 or len(cd8g) == 0:
        raise ValueError(
            f"Markers missing from adata.var_names. Present CD4={cd4g}, CD8={cd8g}. "
            "Check that var_names are gene SYMBOLs (or map from Ensembl first)."
        )

    # score genes (adds columns in ad.obs on the copy)
    sc.tl.score_genes(ad, cd4g, score_name="_cd4_score")
    sc.tl.score_genes(ad, cd8g, score_name="_cd8_score")

    cd4s = ad.obs["_cd4_score"].to_numpy()
    cd8s = ad.obs["_cd8_score"].to_numpy()

    if t_mask is None:
        t_mask = np.ones(ad.n_obs, dtype=bool)
    else:
        t_mask = np.asarray(t_mask, dtype=bool)

    labels = pd.Series(pd.NA, index=ad.obs_names, dtype="object")

    # only label within T-mask; others stay NA
    diff = cd4s - cd8s
    labels.loc[t_mask & (diff >  delta) & (cd4s > min_score)] = "CD4T"
    labels.loc[t_mask & (diff < -delta) & (cd8s > min_score)] = "CD8T"
    labels.loc[t_mask & labels.isna()] = "Ambiguous"

    # write back onto the *original* adata (just obs column in memory)
    adata.obs[out_col] = labels

    # optional: also keep the scores if you want
    adata.obs[out_col + "_cd4_score"] = pd.Series(cd4s, index=adata.obs_names)
    adata.obs[out_col + "_cd8_score"] = pd.Series(cd8s, index=adata.obs_names)

    print(adata.obs[out_col].value_counts(dropna=False))

In [3]:
condition_key = 'condition'
covariate_key = 'patient_id'

control_name = 'Baseline'
stim_name = 'On treatment'

# Read and preprocess adata

In [4]:
data_path = "../../preprocessed_datasets/ICI_original_concat.h5ad"

In [5]:
adata = sc.read_h5ad(data_path)

#adata = adata[adata.obs['disease'].isin(['Pneumonitis', 'No irAE'])]

print(adata.X.max(), adata.X.min())

16688.0 0.0


In [6]:
adata_copy = adata.copy()
adata_copy.var_names = adata_copy.var['gene_symbol'].copy()

t_mask = (adata_copy.obs["ch1__cell type"] == "Human T cells")  
label_cd4_cd8_from_markers(adata_copy, t_mask=t_mask, layer="counts", out_col="cd4_cd8")

/home/SE/miniconda3/envs/trials/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/SE/miniconda3/envs/trials/lib/python3.10/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: Some cells have zero counts
  return fn(*args_all, **kw)


cd4_cd8
Ambiguous    93284
CD4T         93183
CD8T         36211
Name: count, dtype: int64


In [7]:
adata.obs['cell_type'] = adata_copy.obs['cd4_cd8'].tolist()

In [8]:
pd.crosstab(adata.obs['cell_type'], adata.obs['disease'])

disease,Arthritis,Colitis/ Neurotoxicity,No irAE,Pneumonitis,Thyroiditis,Thyroiditis/ Nephritis,Thyroiditis/ Neurotoxicity / Nephritis,Thyroiditis/ Neurotoxicity/ Nephritis
cell_type,,,,,,,,
Ambiguous,17646,2616,18779,11480,41152,376,876,359
CD4T,15429,1437,41780,32332,707,253,1029,216
CD8T,11580,631,11769,9081,2471,149,394,136


In [9]:
adata = adata[adata.obs['cell_type'] == 'CD8T']

In [10]:
adata

View of AnnData object with n_obs × n_vars = 36211 × 20341
    obs: 'sample_id', 'prefix', 'GEO', 'patient_id', 'T', 'title', 'source', 'organism', 'ch1__cell type', 'tissue', 'disease', 'condition', 'library_strategy', 'library_source', 'library_selection', 'cell_type'
    var: 'gene_symbol', 'gene_id_ensembl'

In [11]:
adata.layers['counts'] = adata.X.copy()
adata.raw = adata.copy()
adata.uns = {}

/tmp/ipykernel_498249/3639304367.py:1: ImplicitModificationWarning: Setting element `.layers['counts']` of view, initializing view as actual.
  adata.layers['counts'] = adata.X.copy()


In [12]:
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.filter_genes(adata, min_counts=5)

sc.pp.normalize_total(
    adata, 
    target_sum=1e4, 
    exclude_highly_expressed=True
    )
sc.pp.log1p(adata)

adata.shape, adata.X.max(), adata.X.min()

((35983, 16838), 11.959834, 0.0)

# Create cov_cond column

In [13]:
adata.obs['cov_cond'] = adata.obs[covariate_key].astype(str) + '_' + adata.obs[condition_key].astype(str)

In [14]:
adata.obs['cov_cond']

AAACCTGAGCTTATCG-1-GSM6668925_P1_T      P1_On treatment
AAACCTGCACCTCGTT-1-GSM6668925_P1_T      P1_On treatment
AAACCTGGTTCGTTGA-1-GSM6668925_P1_T      P1_On treatment
AAACCTGTCCTGTACC-1-GSM6668925_P1_T      P1_On treatment
AAACCTGTCTCATTCA-1-GSM6668925_P1_T      P1_On treatment
                                             ...       
TTGCCGTAGTTCGCAT.1-GSM6668972_P24_TN       P24_Baseline
TTGGAACAGTAGGCCA.1-GSM6668972_P24_TN       P24_Baseline
TTGGCAACAAACTGCT.1-GSM6668972_P24_TN       P24_Baseline
TTTATGCCATGCCCGA.1-GSM6668972_P24_TN       P24_Baseline
TTTGTCAGTGGTCTCG.1-GSM6668972_P24_TN       P24_Baseline
Name: cov_cond, Length: 35983, dtype: object

# Compute DEGs

In [15]:
stim_degs = pt.compute_degs(
    adata, 
    cov_key=covariate_key, 
    cond_key=condition_key, 
    stim_name=stim_name, 
    control_name=control_name,
    condition_names=[control_name, stim_name],
    method='wilcoxon'
)

In [16]:
adata.uns[f'rank_genes_groups_{condition_key}'] = {
    stim_name :stim_degs,
}

In [17]:
adata.X.max(), adata.X.min()

(11.959834, 0.0)

In [18]:
adata.X = adata.layers['counts'].copy()

In [19]:
adata.X.max(), adata.X.min()

(9323.0, 0.0)

# Create split column

In [20]:
adata_split = pt.create_split_cols(
    adata=adata, 
    cov_key=covariate_key, 
    cond_key=condition_key, 
    stim_name=stim_name,
    random_state=42
)

# Create cell ids

In [21]:
adata_split.obs['sc_cell_ids'] = list(range(adata.shape[0]))

In [22]:
adata_split.obs['condition'].value_counts()

condition
On treatment    18426
Baseline        17557
Name: count, dtype: int64

In [23]:
adata_split.obs['disease'].value_counts()

disease
No irAE                                   11663
Arthritis                                 11573
Pneumonitis                                9033
Thyroiditis                                2466
Colitis/ Neurotoxicity                      579
Thyroiditis/ Neurotoxicity / Nephritis      387
Thyroiditis/ Nephritis                      148
Thyroiditis/ Neurotoxicity/ Nephritis       134
Name: count, dtype: int64

In [24]:
pd.crosstab(adata_split.obs['condition'],adata_split.obs['disease'])

disease,Arthritis,Colitis/ Neurotoxicity,No irAE,Pneumonitis,Thyroiditis,Thyroiditis/ Nephritis,Thyroiditis/ Neurotoxicity / Nephritis,Thyroiditis/ Neurotoxicity/ Nephritis
condition,,,,,,,,
Baseline,5148,378,5763,4660,1160,61,387,0
On treatment,6425,201,5900,4373,1306,87,0,134


# Write

In [25]:
adata_split.write_h5ad('../../preprocessed_datasets/ici.h5ad')

In [26]:
adata_split

AnnData object with n_obs × n_vars = 35983 × 16838
    obs: 'sample_id', 'prefix', 'GEO', 'patient_id', 'T', 'title', 'source', 'organism', 'ch1__cell type', 'tissue', 'disease', 'condition', 'library_strategy', 'library_source', 'library_selection', 'cell_type', 'n_counts', 'cov_cond', 'split_On treatment_P1', 'split_On treatment_P10', 'split_On treatment_P11', 'split_On treatment_P12', 'split_On treatment_P13', 'split_On treatment_P14', 'split_On treatment_P15', 'split_On treatment_P16', 'split_On treatment_P17', 'split_On treatment_P18', 'split_On treatment_P19', 'split_On treatment_P2', 'split_On treatment_P20', 'split_On treatment_P21', 'split_On treatment_P22', 'split_On treatment_P23', 'split_On treatment_P24', 'split_On treatment_P3', 'split_On treatment_P4', 'split_On treatment_P5', 'split_On treatment_P6', 'split_On treatment_P7', 'split_On treatment_P8', 'split_On treatment_P9', 'sc_cell_ids'
    var: 'gene_symbol', 'gene_id_ensembl', 'n_counts'
    uns: 'log1p', 'rank_genes

In [27]:
adata.obs['patient_id'].value_counts()

patient_id
P10    4365
P3     4313
P9     2714
P1     2470
P2     2400
P7     2390
P24    2138
P8     1611
P18    1570
P17    1482
P12    1376
P5     1339
P20    1158
P4     1127
P11     801
P22     782
P21     753
P19     615
P16     579
P15     521
P13     468
P23     451
P14     412
P6      148
Name: count, dtype: int64

In [28]:
len(adata[adata.obs['disease'] == 'No irAE'].obs['patient_id'].unique())

9

In [29]:
len(adata[adata.obs['disease'] == 'Pneumonitis'].obs['patient_id'].unique())

6

In [30]:
adata[adata.obs['disease'] == 'Pneumonitis'].obs['patient_id'].unique()

['P8', 'P10', 'P11', 'P12', 'P13', 'P14']
Categories (6, object): ['P8', 'P10', 'P11', 'P12', 'P13', 'P14']

In [31]:
len(adata.obs['patient_id'].unique())

24

In [32]:
adata.obs['patient_id'].value_counts()

patient_id
P10    4365
P3     4313
P9     2714
P1     2470
P2     2400
P7     2390
P24    2138
P8     1611
P18    1570
P17    1482
P12    1376
P5     1339
P20    1158
P4     1127
P11     801
P22     782
P21     753
P19     615
P16     579
P15     521
P13     468
P23     451
P14     412
P6      148
Name: count, dtype: int64

In [33]:
pd.crosstab(adata_split.obs['patient_id'],adata_split.obs['condition'])

condition,Baseline,On treatment
patient_id,,
P1,1346,1124
P2,998,1402
P3,2259,2054
P4,478,649
P5,682,657
P6,61,87
P7,545,1845
P8,1009,602
P9,1538,1176


In [34]:
adata_split.X.max(), adata_split.X.min()

(9323.0, 0.0)

In [35]:
adata_split.obs['patient_id'].unique().tolist()

['P1',
 'P2',
 'P4',
 'P5',
 'P7',
 'P17',
 'P3',
 'P8',
 'P9',
 'P10',
 'P11',
 'P18',
 'P19',
 'P6',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24']

In [36]:
['P17','P8','P9','P10','P11','P18','P19','P12','P13','P14','P20','P21','P22','P23','P24']

['P17',
 'P8',
 'P9',
 'P10',
 'P11',
 'P18',
 'P19',
 'P12',
 'P13',
 'P14',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24']

In [43]:
pd.crosstab(
    adata_split.obs['patient_id'], 
    adata_split.obs['disease']
)

disease,Arthritis,Colitis/ Neurotoxicity,No irAE,Pneumonitis,Thyroiditis,Thyroiditis/ Nephritis,Thyroiditis/ Neurotoxicity / Nephritis,Thyroiditis/ Neurotoxicity/ Nephritis
patient_id,,,,,,,,
P1,2470,0,0,0,0,0,0,0
P2,2400,0,0,0,0,0,0,0
P3,4313,0,0,0,0,0,0,0
P4,0,0,0,0,1127,0,0,0
P5,0,0,0,0,1339,0,0,0
P6,0,0,0,0,0,148,0,0
P7,2390,0,0,0,0,0,0,0
P8,0,0,0,1611,0,0,0,0
P9,0,0,2714,0,0,0,0,0
